# NoteBook for IBM capstone project 

#### Import the needed libraries

In [1]:
import pandas as pd
import numpy as np
import folium

#### We can make use of Pandas to read tables in the HTML webpage directly

In [2]:
df_toronto = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [3]:
df_toronto.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


#### Make the first row values as column names

In [4]:
df_toronto.columns = df_toronto.iloc[0]

#### Remove the first row , as it contains only the column names

In [5]:
df_toronto = df_toronto.iloc[1:]

In [6]:
df_toronto.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


#### Remove the non assigned boroughs

In [7]:
df_toronto = df_toronto[df_toronto['Borough'] != 'Not assigned']

In [8]:
df_toronto.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


#### Group the duplicate PostCode m Borough and aggregate the neighbourhood with comma

In [9]:
df_toronto = df_toronto.groupby(['Postcode','Borough'])['Neighbourhood'].agg(','.join).reset_index()

#### set the unassigned neighbourhood value to its borough value

In [10]:
df_toronto.loc[df_toronto['Neighbourhood'] == 'Not assigned','Neighbourhood'] = df_toronto.loc[df_toronto['Neighbourhood'] == 'Not assigned'].Borough

#### Read the co-ordinates file into data frame

In [11]:
df_cord = pd.read_csv('Geospatial_Coordinates.csv')

In [12]:
df_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### left join the toronto detail with co-ordinate csv

In [13]:
df_toronto_cord = df_toronto.merge(df_cord,how='left',left_on='Postcode',right_on='Postal Code')

#### drop the extra postal code column

In [14]:
df_toronto_cord.drop('Postal Code',inplace=True,axis=1)

In [15]:
df_toronto_cord.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
df_toronto_cord.shape

(103, 5)

#### Neighbour analysis

It looks like the bay area has more number of markes clustered together and the other areas are sparsingly clustered.

In [17]:
map_toronto = folium.Map(location=[43.653908, -79.384293], zoom_start=10)

In [18]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_cord['Latitude'], df_toronto_cord['Longitude'], df_toronto_cord['Borough'], df_toronto_cord['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto